In [7]:
import nltk
from nltk.stem.porter import *
import re
import pandas as pd
from textblob import TextBlob
from spacy.lang.en import English
import en_core_web_lg
from  spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
import json
import string

In [8]:
# import spacy
# spacy.require_gpu()

In [9]:
stopwords = list(STOP_WORDS)
punctuations = string.punctuation.replace("#", "")
parser = English()
nlp = en_core_web_lg.load()
# Stemming
stemmer = PorterStemmer()

In [10]:
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    # Lower case all words and strip white spaces
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    # Remove all stop words and punctuations
    tokens = [word for word in tokens if word not in stopwords and word not in punctuations]
    return tokens

In [11]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [38]:
def clean_sentence(sentence):
    doc = nlp(sentence)
    
    # Token matchers
    matcher = Matcher(nlp.vocab)
    
    # Create matcher to detect urls
    pattern = [{ "LIKE_URL": True }]
    matcher.add("UrlDetection", None, pattern)
    
    # Create matcher to detect ...
    pattern = [{ "TEXT": "..." }]
    matcher.add("MoreDotsDetection", None, pattern)

    matches = matcher(doc)
    
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        sentence = sentence.replace(span.text, '')
    
    # Tokenize sentence and join
    sentence = ' '.join(str(token) for token in spacy_tokenizer(sentence))
    
    # Remove twitter handles
    sentence = remove_pattern(sentence, "@[\w]*")
    
    sentence = '#'.join([phrase for phrase in [e[1:] for e in (' ' + sentence).split("#")]])

    # remove words with length less than 3 and not #
    sentence = ' '.join([word for word in sentence.split() if word[0] == '#' or len(word)>3])
    
    return sentence

clean_sentence("The latest Logan's <mark>marktag</mark> DX Daily #AD (Cloud, AI/ML, Analytics & IoT)! https://t.co/Ac3cKz73Gx Thanks to @JD_Corporate... https://t.co/OOyzLIPxA2")

'late logan mark marktag</mark daily #ad cloud analytics thanks'

In [64]:
# DON'T RUN THIS
# new_data = { 'results': [] }

# with open('new_posts.txt', encoding="utf-8") as json_file:
#     new_data = json.load(json_file)

# with open('posts.txt', encoding="utf-8") as read_file:
#     data = json.load(read_file)
#     new_data['results'] = new_data['results'] + data['results']
    
#     print(len(new_data['results']))
#     with open('new_posts.txt', 'w') as out_file:
#         json.dump(new_data, out_file)

29871


In [42]:
# DON'T RUN THIS
# with open('new_posts.txt', encoding="utf-8") as json_file:
#     new_data = json.load(json_file)
    
# new_data = [result['title'] for result in new_data['results']]

# print(new_data[0:5])

['Love this by Jim Whitehurst <mark>@RedHat</mark> &lt; "Instead of being the commander-in-chief, the leader became the communicator-... https://t.co/gYcCe5NXvI', 'Using Keycloak instead of Picketlink for SAML-based authentication #RedHat https://t.co/XavswHIz9q', 'DevNation Live: Kubernetes enterprise integration patterns with Camel K #RedHat https://t.co/vSbE3Tgt1q', 'Die #RedHat Innovation Awards 2020 sind eroffnet. Nutzt ihr Open-Source-Losungen von Red Hat auf innovative Weise?... https://t.co/Q2MnnjOYMb', '#RedHat #OpenStack Platform remains a strategic piece of our #hybridcloud vision, bridging the gap between existing... https://t.co/qQ8VBT9OpI']


In [43]:
df = pd.DataFrame(columns=('Text', 'Label'))

In [44]:
def create_df(texts):
    tw = []
    pl = []

    for text in texts:
        text = clean_sentence(text)
        str(text)
        sent  = TextBlob(text)
        tw.append(text)
        pl.append(sent.sentiment.polarity)

    df['Text'] = tw
    df['Label'] = pl

    return df

In [45]:
create_df(new_data)

df.shape

(29871, 2)

In [48]:
df.head()

,Text,Label
0,love whitehurst mark></mark instead commander ...,0.500000
1,keycloak instead picketlink saml base authenti...,-0.800000
2,devnation live kubernetes enterprise integrati...,0.136364
3,#redhat innovation awards 2020 sind eroffnet n...,0.250000
4,#redhat #openstack platform remain strategic p...,0.000000


In [49]:
def sentiment_type(num):
    if (num >= -1 and num < 0):
        return 'NEGATIVE'
    elif (num > 0 and num <= 1):
        return 'POSITIVE'
    else:
        return 'NEUTRAL'

In [50]:
df['Label'] = df['Label'].apply(lambda x: sentiment_type(x))

In [51]:
df.head()

,Text,Label
0,love whitehurst mark></mark instead commander ...,POSITIVE
1,keycloak instead picketlink saml base authenti...,NEGATIVE
2,devnation live kubernetes enterprise integrati...,POSITIVE
3,#redhat innovation awards 2020 sind eroffnet n...,POSITIVE
4,#redhat #openstack platform remain strategic p...,NEUTRAL


In [52]:
# Remove duplicates
df = df.drop_duplicates(subset=['Text'], keep='first')

In [53]:
df.to_csv("data/large_tech.csv", sep='\t', encoding='utf-8')

In [54]:
df = pd.read_csv("data/large_tech.csv", sep='\t', encoding='utf-8')

In [55]:
df = df[['Text', 'Label']]
df.head()

,Text,Label
0,love whitehurst mark></mark instead commander ...,POSITIVE
1,keycloak instead picketlink saml base authenti...,NEGATIVE
2,devnation live kubernetes enterprise integrati...,POSITIVE
3,#redhat innovation awards 2020 sind eroffnet n...,POSITIVE
4,#redhat #openstack platform remain strategic p...,NEUTRAL


In [56]:
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [57]:
def main(model=None, output_dir=None, n_iter=20, n_texts=2000):
    if model is not None:
        # load existing spaCy model
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
    else:
        # Create blank Language class
        nlp = spacy.load('en_core_web_lg')
        print("Created new model")

    # Add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'textcat' not in nlp.pipe_names:
        textcat = nlp.create_pipe('textcat')
        nlp.add_pipe(textcat, last=True)
    # Otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe('textcat')

    # Add label to text classifier
    textcat.add_label('POSITIVE')
    textcat.add_label('NEGATIVE')
    textcat.add_label('NEUTRAL')

    (train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)
    print("Using {} examples ({} training, {} evaluation)"
          .format(n_texts, len(train_texts), len(dev_texts)))
    train_data = list(zip(train_texts,
                          [{'cats': cats} for cats in train_cats]))

    # Get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    
    # Only train textcat by disabling other pipes
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        print("Training the model...")
        print('{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('No.', 'LOSS', 'P', 'R', 'F'))
        for i in range(n_iter):
            losses = {}
            # Batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                print(losses)
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                           losses=losses)
            
            with textcat.model.use_params(optimizer.averages):
                # Evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print('{0}\t{1:.3f}\t{2:.3f}\t{3:.3f}\t{4:.3f}'  # print a simple table
                  .format(i, losses['textcat'], scores['textcat_p'],
                          scores['textcat_r'], scores['textcat_f']))

    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [58]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [59]:
def load_data(limit=0, split=0.8):
    # Partition off part of the train data for evaluation
    cats = []
    for y in df['Label']:
        if (y == 'NEGATIVE'):
            cats.append({ 'POSITIVE': 0, 'NEGATIVE': 1, 'NEUTRAL': 0})
        elif (y == 'POSITIVE'):
            cats.append({ 'POSITIVE': 1, 'NEGATIVE': 0, 'NEUTRAL': 0})
        else:
            cats.append({ 'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 1})

    split = int(len(df['Text'].values) * split)
    return (df['Text'][:split], cats[:split]), (df['Text'][split:], cats[split:])

In [ ]:
main(model=None, output_dir="text_cnn_models/en_large_model", n_iter=20, n_texts=19592)

In [ ]:
model = 'text_cnn_models/en_large_model'

nlp = spacy.load(model)

def predict_test(text):
    test_text = clean_sentence(text)
    doc = nlp(test_text)
    print(test_text, doc.cats)